In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
df_closed = pd.read_csv("../data/raw/olist_closed_deals_dataset.csv")
#
df_mkt = pd.read_csv("../data/raw/olist_marketing_qualified_leads_dataset.csv")

In [5]:
df_mkt_closed = df_mkt.merge(df_closed, on='mql_id', how='left')
#
df_processed = df_mkt_closed.assign(
        first_contact_date=lambda df: pd.to_datetime(df_mkt_closed['first_contact_date'], format='%Y-%m-%d', errors='coerce'),
        won_date= pd.to_datetime(df_mkt_closed['won_date'].str[:10], format='%Y-%m-%d', errors='coerce'),)
#
df_processed['contact_period'] = df_processed['first_contact_date'].astype(str).str[:7]

In [10]:
mql_daily_series = df_processed.groupby('first_contact_date', as_index=False).agg(mql_count=('mql_id', 'count'))
#
mql_period_series = df_processed.groupby('contact_period', as_index=False).agg(mql_count=('mql_id', 'count'))
#
mql_series = df_processed.groupby(['first_contact_date', 'contact_period'], as_index=False).agg(mql_count=('mql_id', 'count'))

In [7]:
px.line(mql_daily_series, x='first_contact_date', y='mql_count', title='MQLs por día')

In [8]:
px.line(mql_period_series, x='contact_period', y='mql_count', title='MQLs por día')

In [11]:
px.box(mql_series, x='contact_period', y='mql_count')